<a href="https://colab.research.google.com/github/greyhound101/multilingial/blob/master/portuguese_multilingual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Mounted at /content/gdrive


In [ ]:
pip install sentencepiece 

In [ ]:
pip install transformers

In [ ]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  29234      0 --:--:-- --:--:-- --:--:-- 29234
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-nightly ...
Uninstalling torch-1.8.0a0+unknown:
  Successfully uninstalled torch-1.8.0a0+unknown
Uninstalling torchvision-0.9.0a0+7dfd8dc:
  Successfully uninstalled torchvision-0.9.0a0+7dfd8dc
Copying gs://tpu-pytorch/wheels/torch-nightly-cp36-cp36m-linux_x86_64.whl...
- [1 files][123.1 MiB/123.1 MiB]                                                
Operation completed over 1 objects/123.1 MiB.                                    
Copying gs://tpu-pytorch/wheels/torch_xla-nightly-cp36-cp36m-linux_x86_64.whl...
\ [1 files][132.0 MiB/132.0 MiB]                                                
Operation completed over 1 objects/132.0 MiB.                                    
Copying g

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors



def regular_encode(texts, tokenizer, maxlen=192):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])
def fl():
    def focal_loss(targets, inputs):
        
        one=(1-targets)*K.log(1-inputs)
        zero=(targets*K.log(inputs))
        loss = K.mean((one+zero)*-1)

        return loss
    return focal_loss
def build_model(transformer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss=fl(), metrics=['accuracy'])
    
    return model

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None
if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Running on TPU  grpc://10.67.114.218:8470
INFO:tensorflow:Initializing the TPU system: grpc://10.67.114.218:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.67.114.218:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [ ]:
AUTO = tf.data.experimental.AUTOTUNE


# Configuration
EPOCHS = 2
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 192
MODEL = 'jplu/tf-xlm-roberta-large'

In [ ]:

# First load the real tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL)



In [ ]:
train = pd.read_csv("/content/gdrive/My Drive/multilingual/jigsaw-toxic-comment-train-google-tr-cleaned.csv.zip",usecols=['comment_text','toxic'])
train=train.dropna()
valid = pd.read_csv('/content/gdrive/My Drive/multilingual/validation.csv.zip')
test = pd.read_csv('/content/gdrive/My Drive/multilingual/test.csv.zip')
sub = pd.read_csv('/content/gdrive/My Drive/multilingual/sample_submission.csv')
pre=pd.read_csv('/content/gdrive/My Drive/multilingual/submission.csv')

In [ ]:


test['toxic']=pre['toxic']
test=test[['content','toxic']]
test.columns=['comment_text','toxic']
total=pd.concat([test,train,valid[['comment_text','toxic']]]).sample(frac=1).reset_index(drop=True)
total.head()



,comment_text,toxic
0,Engellendim mi? Gerçekten mi? Emin misiniz?,0.000000
1,"""\n\n 2006 temizliği\n\nMerhaba Yboy. 2006 Com...",0.000000
2,""":::: Gerçekten iyi bir gösteri olmalı. Cumart...",0.000000
3,Bireysel wikipedia'lara atıfta bulunan başka ş...,0.000000
4,Önemli değil. Erdal da bazen sert davranır ama...,0.005132


In [ ]:
x_train = regular_encode(list(total.comment_text.values), tokenizer, maxlen=192)
x_test = regular_encode(list(test.comment_text.values), tokenizer, maxlen=192)
y_train = total.toxic.values.astype(float)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2143: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
import keras.backend as K
test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE))

In [ ]:
from sklearn.model_selection import KFold
from transformers import AutoTokenizer
import keras.backend as K 
import gc
kf=KFold(n_splits=5, random_state=42)
for en,(train,test) in enumerate(kf.split(range(x_train.shape[0]))):
    
    train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train[train], y_train[train]))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO))
    
    
    with strategy.scope():
        transformer_layer=TFAutoModel.from_pretrained(MODEL)
        model=build_model(transformer_layer,192)
    n_steps = train.shape[0] // BATCH_SIZE
    
    
    train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    epochs=EPOCHS)
    pre = model.predict(test_dataset, verbose=1)
    np.save('/content/gdrive/My Drive/multilingual/predictions_'+str(en)+'_xlm.npy', pre)
    del([train_dataset,model])
    gc.collect()

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Some layers from the model checkpoint at jplu/tf-xlm-roberta-large were not used when initializing TFXLMRobertaModel: ['lm_head']
- This IS expected if you are initializing TFXLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLMRobertaModel were initialized from the model checkpoint at jplu/tf-xlm-roberta-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7fc0baeced90> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).WARNING:tensorflow:AutoGraph could not transform <bound method Socket.send of <zmq.sugar.socket.Socket object at 0x7fc090c60590>> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7fc0baeced90> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7fc0baeced90> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


Epoch 1/2


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:430: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 256002048 elements. This may consume a large amount of memory.
  num_elements)
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
